In [6]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df_original = pd.read_csv('/content/drive/MyDrive/CO544/transformed_rainfall_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Assuming your dataframe is named 'df'
df = df_original[~df_original.index.isin([0, 1])]

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming your data is loaded into a DataFrame named 'df'

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Extracting day of the year as a feature
df['DayOfYear'] = df['Date'].dt.dayofyear

# Select features and target
features = ['DayOfYear']
targets = ['Vavuniya', 'Anuradhapura', 'Maha Illuppallama']

# Feature engineering

# Lag features (previous day's value)
for target in targets:
  df[f'{target}_lag1'] = df[target].shift(1)

# Rolling mean (weekly window)
window_size = 7  # You can experiment with different window sizes
for target in targets:
  df[f'{target}_rolling_mean_{window_size}'] = df[target].rolling(window=window_size).mean()

# Month and Weekday features
# df['Month'] = df['Date'].dt.month
# df['Is_Weekend'] = np.where(df['Date'].dt.weekday_name.isin(['Saturday', 'Sunday']), 1, 0)

# After feature engineering
df.dropna(inplace=True)  # Drop rows with any NaN values

# Splitting the data into training and testing sets
X = df[features + [col for col in df.columns if col.endswith('_lag1') or col.endswith(f'_rolling_mean_{window_size}')]]  # Include engineered features
y_vavuniya = df['Vavuniya']
y_anuradhapura = df['Anuradhapura']
y_maha = df['Maha Illuppallama']



X_train, X_test, y_train_v, y_test_v = train_test_split(X, y_vavuniya, test_size=0.2, random_state=42)
X_train, X_test, y_train_a, y_test_a = train_test_split(X, y_anuradhapura, test_size=0.2, random_state=42)
X_train, X_test, y_train_m, y_test_m = train_test_split(X, y_maha, test_size=0.2, random_state=42)

# ... (rest of your code for training and prediction)


<ipython-input-8-ab9b7845a373>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
<ipython-input-8-ab9b7845a373>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DayOfYear'] = df['Date'].dt.dayofyear
<ipython-input-8-ab9b7845a373>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [11]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

# Perform Grid Search for each target
grid_search_v = GridSearchCV(estimator=dtr_v, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_v.fit(X_train, y_train_v)

grid_search_a = GridSearchCV(estimator=dtr_a, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_a.fit(X_train, y_train_a)

grid_search_m = GridSearchCV(estimator=dtr_m, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_m.fit(X_train, y_train_m)

# Get the best parameters
best_params_v = grid_search_v.best_params_
best_params_a = grid_search_a.best_params_
best_params_m = grid_search_m.best_params_

# Use the best parameters to initialize the final model
dtr_v = DecisionTreeRegressor(**best_params_v)
dtr_a = DecisionTreeRegressor(**best_params_a)
dtr_m = DecisionTreeRegressor(**best_params_m)

# Fit the models with the best parameters
dtr_v.fit(X_train, y_train_v)
dtr_a.fit(X_train, y_train_a)
dtr_m.fit(X_train, y_train_m)

# Predict
y_pred_v = dtr_v.predict(X_test)
y_pred_a = dtr_a.predict(X_test)
y_pred_m = dtr_m.predict(X_test)

# Evaluate the model
mse_v = mean_squared_error(y_test_v, y_pred_v)
mse_a = mean_squared_error(y_test_a, y_pred_a)
mse_m = mean_squared_error(y_test_m, y_pred_m)

print(f'Decision Tree Regressor MSE for Vavuniya: {mse_v}')
print(f'Decision Tree Regressor MSE for Anuradhapura: {mse_a}')
print(f'Decision Tree Regressor MSE for Maha Illuppallama: {mse_m}')


Fitting 3 folds for each of 144 candidates, totalling 432 fits
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Decision Tree Regressor MSE for Vavuniya: 124.60631420739274
Decision Tree Regressor MSE for Anuradhapura: 160.8274326188902
Decision Tree Regressor MSE for Maha Illuppallama: 135.00125369961876


In [18]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid (same as before)
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

# Create a dictionary to store best models and parameters
best_models = {}
best_params = {}

# Loop through each target variable
target_variables = ['Vavuniya', 'Anuradhapura', 'Maha Illuppallama']
for target in target_variables:
  # Separate data for the current target
  X_train, X_test, y_train, y_test = train_test_split(X, df[target], test_size=0.2, random_state=42)

  # Create a DecisionTreeRegressor instance
  dtr = DecisionTreeRegressor(random_state=42)

  # Perform Grid Search for the current target
  grid_search = GridSearchCV(estimator=dtr, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
  grid_search.fit(X_train, y_train)

  # Store best parameters and model for this target
  best_params[target] = grid_search.best_params_
  best_models[target] = grid_search.best_estimator_

# Use the best parameters to create final models for each target
for target, params in best_params.items():
  dtr = DecisionTreeRegressor(**params)
  dtr.fit(X_train, y_train)
  best_models[target] = dtr

# Make predictions and evaluate MSE for each target (similar to previous code)
for target, model in best_models.items():
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print(f'Decision Tree Regressor MSE for {target}: {mse}')


Fitting 3 folds for each of 144 candidates, totalling 432 fits
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Decision Tree Regressor MSE for Vavuniya: 110.16263101048482
Decision Tree Regressor MSE for Anuradhapura: 149.40950080374398
Decision Tree Regressor MSE for Maha Illuppallama: 127.03648942550703
